원문: Moler, Cleve B. [Numerical computing with MATLAB](https://www.mathworks.com/moler/chapters.html). Society for Industrial and Applied Mathematics, 2008.

*최소자승법*은 과결정되어 있거나 부정확하게 명세된 연립방정식의 근을 찾는 데 자주 사용되는 방법이다.  엄밀해를 구하는 대신, 우리는 오차의 제곱의 합을 최소화하는 것 만을 목표로 한다

최소자승 조건은 중요한 통계적 해석을 가진다. 만일 아래에 깔린 오류 분표에 대한 적당한 확률적 가정이 주어지면, 최소자승법은 매개변수의 *최대 가능도* 추정값을 계산해낼 수 있다. 심지어 확률적 가정이 만족되지 않더라도, 다년간의 경험에 따르면, 최소자승법은 유용한 결과값을 계산해 낸다.

최소 자승 문제를 위한 계산 기법은 직교 행렬 분해를 이용한다.